In [2]:
import pandas as pd
import geopandas as gpd
import os
import numpy as np

this notebooks creates gpkgs for each day with all of its road closures

functions

In [3]:
def update_km(closures):
    for index, row in closures.iterrows():
        #update the comma-separated km markings to be a list
        km = row['KM']
        if isinstance(km, str):
            km = km.split(',')
        col = closures.columns.get_loc('KM')
        closures.iat[index, col] = km
    return closures

In [4]:
def add_closures(closures):
    road_sample = gpd.read_file('./road-data/54.gpkg')
    date_gpkg = road_sample[0:0]
    
    #now we want to manually update the closures and stuff
    for index, row in closures.iterrows():
        road_id = row['Road ID']
        km = row['KM']

        #find the road and date files, update
        road_path = './road-data/' + str(road_id) + '.gpkg'
        road_gpkg = gpd.read_file(road_path)
        status_col = road_gpkg.columns.get_loc('status')

        #only 1 closure
        if isinstance(km, int):
            try:
                row_ind = road_gpkg.index[road_gpkg['segment_id'] == km].tolist()
                road_gpkg.at[row_ind, 'status'] = 1
                date_gpkg = date_gpkg.append(road_gpkg)

            except IndexError:
                print("Not a valid road closure - road does not have this km marking")
                continue

        else:
            #print(road_id)
            for i in range(0, len(km)):
                try:
                    row_ind = road_gpkg.index[road_gpkg['segment_id'] == int(km[i])].tolist()
                    road_gpkg.at[row_ind, 'status'] = 1
                except IndexError:
                    print("Not a valid road closure - road does not have this km marking")
            date_gpkg = date_gpkg.append(road_gpkg)
    
    #date = closures['Date'][0]
    date = closures['Date'][0].date().strftime('%Y-%m-%d')
    date_gpkg.loc[:, 'date'] = date
    
    return date_gpkg

In [5]:
def make_closure_file(filename):
    closures = pd.read_excel(filename) #list of road closures on a date
    closures.astype(object)

    date = closures['Date'][0].date().strftime('%Y-%m-%d')
    date_path = './closure-data/' + date + '.gpkg'

    closures = update_km(closures)

    date_gpkg = add_closures(closures)
    date_gpkg.to_file(date_path, driver='GPKG')
    print('done!')
    return date_gpkg

In [6]:
data = make_closure_file('./daily-data/2021-06-23.xlsx')

done!


In [7]:
data.loc[data['status']==1]

,road_id,date,status,segment_id,geometry
33,54,2021-06-23,1,34,"MULTILINESTRING ((79.16107 30.18452, 79.16089 ..."
56,54,2021-06-23,1,57,"MULTILINESTRING ((79.07799 30.17550, 79.07768 ..."
19,57,2021-06-23,1,20,"LINESTRING (78.86945 30.16930, 78.86972 30.169..."
6,156,2021-06-23,1,7,"LINESTRING (79.16940 30.50643, 79.16938 30.506..."
21,156,2021-06-23,1,22,"LINESTRING (79.09142 30.52461, 79.09135 30.524..."
1,135,2021-06-23,1,2,"LINESTRING (79.51677 30.06170, 79.51688 30.061..."
46,135,2021-06-23,1,47,"LINESTRING (79.62267 30.19491, 79.62275 30.194..."


In [8]:
#to make sure things saved correctly
data2 = gpd.read_file('./closure-data/2021-06-23.gpkg')

In [9]:
data2.loc[data2['status'] ==1]

,road_id,date,status,segment_id,geometry
33,54,2021-06-23,1,34,"MULTILINESTRING ((79.16107 30.18452, 79.16089 ..."
56,54,2021-06-23,1,57,"MULTILINESTRING ((79.07799 30.17550, 79.07768 ..."
96,57,2021-06-23,1,20,"LINESTRING (78.86945 30.16930, 78.86972 30.169..."
142,156,2021-06-23,1,7,"LINESTRING (79.16940 30.50643, 79.16938 30.506..."
157,156,2021-06-23,1,22,"LINESTRING (79.09142 30.52461, 79.09135 30.524..."
161,135,2021-06-23,1,2,"LINESTRING (79.51677 30.06170, 79.51688 30.061..."
206,135,2021-06-23,1,47,"LINESTRING (79.62267 30.19491, 79.62275 30.194..."
